In [ ]:
import pandas as pd
import numpy as np

Insert all GPI data with the corresponding fips_10_4 codes as created in jupyter file named "Dataframe_GPI_score_fips_10_4_codes_journal."

In [ ]:
gpifips = pd.read_csv('data/Gpi_fips_10_4.csv')

In [ ]:
gpifips.head(3)

Delete the columns with the full country names

In [ ]:
del gpifips['Country']

Transpose the dataframe so that I have one row per each country and year.<br>  

In [ ]:
Ngpifips = gpifips.set_index('FIPS_10_4', append=True).stack().reset_index().drop('level_0',axis=1)

Rename columns

In [ ]:
Ngpifips.columns=['country','year','gpi_score']

In [ ]:
Ngpifips.head(1)

In [ ]:
countries=Ngpifips['country'].unique()
countries

Control whether year data are datetime data that we need for the interpolation

In [ ]:
Ngpifips.year.dtype

They are object data, so let's convert them to datetime.

In [ ]:
Ngpifips['year'] = pd.to_datetime(Ngpifips['year'])

In [ ]:
Ngpifips.year.dtype

Verify whether my machine is accepting '<M8[ns]' as datetime

In [ ]:
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')

### Interpolation for each country

**Upsample (= When you increase the frequency of the samples, such as from minutes to seconds. VS downsample**) the countries' database in order to create nans and then interpolate it.<br>

In [ ]:
for c in countries:
    single_country = Ngpifips.loc[Ngpifips.country == c] #loc is looking for location (if I try to print single country and break it gives me the first country dataset that creates)
    single_country = single_country.drop('country',axis=1)
    single_country.set_index('year', inplace = True) #I set the year as the index since this way the data should be inserted for interpolation#(inplace=True)->it modifies the initial dataframe without needing to rename it
    upsampled = single_country.resample('MS') #Upsample per month in order to create nan gpi values for the rest of the 11 months that are needed for the interpolation
    gpi_interp = upsampled.interpolate(method='linear')#Interpolate linearly
    firstdate = gpi_interp.index[0]#get the first date to help create a new index with extra dates
    extradates = pd.date_range(firstdate, periods = len(gpi_interp)+2 , freq='MS') #create the index column with the dates and add the two extra dates to reach the last month which is March
    gpi_interp = gpi_interp.reindex(extradates) #change the index column with the new index column
    shifted = gpi_interp.shift(2).dropna() #I give the GPI yearly value to the month of March
    newind = shifted.index.strftime('%Y%m') #Turn the index from YYMMDD to YYMM, so that I can easily merge later.
    shifted.index = newind #Pass the new index format to the dataframe.
    shifted.reset_index(inplace=True)#Reset the index so that avoid having MonthYear column as an index.
    shifted.columns = ['MonthYear','GPI_score']
    shifted.to_csv('data/interpolated_gpi/interpolated_gpi_%s.csv' %c,index=False)  